In [ ]:
pip install pyspark


In [ ]:
pip install findspark


In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression

In [ ]:
spark = SparkSession.builder.appName("student performance").getOrCreate()

In [ ]:
dataset = spark.read.csv("/content/Student_performance_data _.csv", inferSchema = True, header = True)
print('There are ',len(dataset.columns),'columns in this dataset')
print('There are ',dataset.count(), 'rows in this dataset')


In [ ]:
dataset.show()
# dataset previously cleaned before using so this step is skipped, straight to analysis

In [ ]:
assembler = VectorAssembler(inputCols = ['Age', 'Gender', 'Ethnicity', 'ParentalEducation', 'StudyTimeWeekly', 'Absences', 'Tutoring', 'ParentalSupport', 'Extracurricular','Sports','Music','Volunteering'], outputCol='features')
output = assembler.transform(dataset)

In [ ]:
finalised_data = output.select('features', 'GradeClass')
finalised_data.show()

In [ ]:
train, test = finalised_data.randomSplit([0.8, 0.2])

In [ ]:
log_reg = LogisticRegression(featuresCol = "features", labelCol="GradeClass")
log_regmodel = log_reg.fit(train)
predictions = log_regmodel.transform(test)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
multi_eval = MulticlassClassificationEvaluator(labelCol="GradeClass", predictionCol="prediction")
accuracy = multi_eval.evaluate(predictions, {multi_eval.metricName: "accuracy"})
precision = multi_eval.evaluate(predictions, {multi_eval.metricName: "weightedPrecision"})
recall = multi_eval.evaluate(predictions, {multi_eval.metricName: "weightedRecall"})
f1 = multi_eval.evaluate(predictions, {multi_eval.metricName: "f1"})

In [ ]:
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1: {f1:.4f}")
